# Advanced py.test Fixtures

```
Author: Floris Bruynooghe - 2014
Email: flub@devork.be 
Touchup: Ronny Pfannschmidt - 2019
Email: rpfannsc@redhat.com
```

## Introduction

Fixtures are powerful:

- Dependency Injection
- Isolated
- Composable

Assuming you know:
- pytest
- basic fixtures


- floris: Been using and contributing to py.test for a few years
- Use at own risk
- Some patterns we use in ~70k source
- Some from pytest-django
- Some made up stuff


## The Basics

You should know this:

In [1]:
%%writefile test_simple_fixture_in_class.py

import pytest
  
@pytest.fixture
def foo():
    return 42
  
def test_foo(foo):
    assert foo == 42


class TestBar:
  
    @pytest.fixture
    def bar(self, request):
        print("\nSetup of fixture bar")
        yield 7
        print("\nTeardown of fixture bar")

    def test_bar(self, foo, bar):
        assert foo != bar

Overwriting test_simple_fixture_in_class.py


In [2]:
!pytest test_simple_fixture_in_class.py -sv

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0 -- /home/rpfannsc/Projects/pytest-dev/pytest-talks/.env/bin/python3.7
cachedir: .pytest_cache
rootdir: /home/rpfannsc/Projects/pytest-dev/pytest-talks/talks, inifile: pytest.ini
collected 2 items                                                              

test_simple_fixture_in_class.py::test_foo PASSED
test_simple_fixture_in_class.py::TestBar::test_bar 
Setup of fixture bar
PASSED
Teardown of fixture bar


=========================== 2 passed in 0.03 seconds ===========================


- Dependency Injection!
- Hope you know this

## Caching fixtures

* Fixture decorator has scope argument
* Available scopes: function, class, module, session

In [3]:
%%writefile test_simple_scope_cache.py
import pytest

@pytest.fixture(scope='session')
def foo(request):
    print('\nsession setup')
    yield foo
    print('\nsession finalizer')
  

@pytest.fixture(scope='function')
def bar(request):
    print('\nfuntion setup')
    yield "bar"
    print('\nfunction finalizer')

    
def test_one(foo, bar):
    pass
  
def test_two(foo, bar):
    pass


Overwriting test_simple_scope_cache.py


In [4]:
! py.test test_simple_scope_cache.py -s -v 

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0 -- /home/rpfannsc/Projects/pytest-dev/pytest-talks/.env/bin/python3.7
cachedir: .pytest_cache
rootdir: /home/rpfannsc/Projects/pytest-dev/pytest-talks/talks, inifile: pytest.ini
collected 2 items                                                              

test_simple_scope_cache.py::test_one 
session setup

funtion setup
PASSED
function finalizer

test_simple_scope_cache.py::test_two 
funtion setup
PASSED
function finalizer

session finalizer


=========================== 2 passed in 0.03 seconds ===========================



- session only setup once
- fuction twice
- `-s~` does not capture stdout

## Interdependent fixtures

Fixture can use fixtures too:

In [5]:
import pytest

@pytest.fixture(scope='session')
def db_conn():
    return ...
  
@pytest.fixture(scope='module')
def db_table(request, db_conn):
    table = create_table(db_conn, 'foo')
    yield table  
    drop_table(db_conn, table)
    
def test_bar(db_table):
    pass

- Fixtures can depend on each other
- Functional test example
- Request is just a built-in fixture

## Skip/fail in fixture

Fixtures can trigger skipping/failing of all dependent tests:

In [6]:
import pytest
import redis

@pytest.fixture(scope='session')
def redis_client():
    servers = ['localhost', 'venera.clockhouse']
    for hostname in servers:
        try:
            return redis.StrictRedis(hostname)
        except redis.ConnectionError:
            continue
    else:
        pytest.skip('No Redis server found')


- this will respect scope
- also pytest.fail()
- .fail() and .skip() just raise an exception


## Marks

Rember tests can be marked:

In [7]:
%%writefile test_have_markers.py
import pytest

@pytest.mark.mymarker
@pytest.mark.other_marker
def test_something():
    pass


Overwriting test_have_markers.py


### Run tests based on markers:

In [8]:
! pytest -m "not mymarker" test_have_markers.py

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0
rootdir: /home/rpfannsc/Projects/pytest-dev/pytest-talks/talks, inifile: pytest.ini
collected 1 item / 1 deselected                                                

========================= 1 deselected in 0.02 seconds =========================


### Make them known:

In [9]:
%%writefile pytest.ini
[pytest]
markers =
  mymarker: a custom marker
  other_marker: another marker
  linux: helper marker for checking the platform

Overwriting pytest.ini


- mark a test
- run test from command line

## Using Marks from Fixtures
  

In [10]:
import pytest

@pytest.fixture
def mongo_client(request):
    marker = request.node.get_marker('mongo_db') or pytest.mark.not_used
    def apifun(db = 'TestDB'):
        return db
    db = apifun(*marker.args, **marker.kwargs)
    return pymongo.MongoClient('127.0.0.1/{}'.format(db))
  
@pytest.mark.mongo_db('Users')
def test_something(mongo_client):
    pass


- Use a mark as a parameter to a fixture
- Maybe consider re-designing the fixture
- apifun current hack to get python argument parsing

## Autouse fixtues

Setup/teardown without explicit request:

In [11]:
%%writefile test_markers_conditional_skip_autouse.py
import pytest
import platform
import attr

@attr.s
class MemSizes:
    stack = attr.ib(default=42)


@pytest.mark.linux
def test_mem_stack():
    assert MemSizes().stack == 42
  
@pytest.fixture(autouse=True)
def _platform_skip(request):
    marker = request.node.get_closest_marker('linux')
    if marker and platform.system() != 'Linux':
        pytest.skip('N/A on {}'.format(platform.system()))

Overwriting test_markers_conditional_skip_autouse.py


In [12]:
! pytest test_markers_conditional_skip_autouse.py

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0
rootdir: /home/rpfannsc/Projects/pytest-dev/pytest-talks/talks, inifile: pytest.ini
collected 1 item                                                               

test_markers_conditional_skip_autouse.py .                               [100%]

=========================== 1 passed in 0.03 seconds ===========================


- An autouse fixture to detect the mark
- Autouse fixture invoked before each test
- Autouse also useful without marks


## Parametrizing fixtures

- Individual fixtures can be paremeterised
- Multiple parameterised fixtures combine

In [13]:
%%writefile combinatoric_parameters.py
import pytest
import socket


def create_db_uri(kind):
    return kind + "://localhost"

    
@pytest.fixture(params=['ora', 'pg', 'sqlite'])
def dburi(request):
    return create_db_uri(request.param)
  
@pytest.fixture(params=['ipv4', 'ipv6'])
def addr_family(request):
    return socket.AF_INET if request.param == 'ipv4' else socket.AF_INET6
  

Overwriting combinatoric_parameters.py


In [1]:
%%writefile test_combinatoric_parameters.py

import attr
import socket

@attr.s
class MyObj:
    uri = attr.ib()
    addr_family = attr.ib(default=socket.AF_INET)
    
    def it_works(self):
        return True
    transaction_works = it_works

def test_txn(dburi):
    inst = MyObj(dburi)
    assert inst.transaction_works()

    
def test_conn(dburi, addr_family):
    inst = MyObj(dburi, addr_family)
    assert inst.it_works()

Overwriting test_combinatoric_parameters.py


In [2]:
!PYTHONPATH=. pytest test_combinatoric_parameters.py -p combinatoric_parameters -v

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0 -- /home/rpfannsc/Projects/pytest-dev/pytest-talks/.env/bin/python3.7
cachedir: .pytest_cache
rootdir: /home/rpfannsc/Projects/pytest-dev/pytest-talks/talks, inifile: pytest.ini
collected 9 items                                                              

test_combinatoric_parameters.py::test_txn[ora] PASSED                    [ 11%]
test_combinatoric_parameters.py::test_txn[pg] PASSED                     [ 22%]
test_combinatoric_parameters.py::test_txn[sqlite] PASSED                 [ 33%]
test_combinatoric_parameters.py::test_conn[ora-ipv4] PASSED              [ 44%]
test_combinatoric_parameters.py::test_conn[ora-ipv6] PASSED              [ 55%]
test_combinatoric_parameters.py::test_conn[pg-ipv4] PASSED               [ 66%]
test_combinatoric_parameters.py::test_conn[pg-ipv6] PASSED               [ 77%]
test_combinatoric_parameters.p


- Powerful, combinatory
- Functional test example
- This will execute 6 versions of test_func


## Skipping Parameters

Skipping can be done on a parameter level:

In [16]:
import pytest
try:
    import cx_Oracle as ora
except ImportError:
    ora = None
  

needs_ora = pytest.mark.skipif(ora is None, reason='No Oracle installed')
  
@pytest.fixture(params=[
    'pg',
    pytest.param(ora, marks=needs_ora),
])
def dburi(request):
    return create_db_uri(request.param)


- Marks can be assigned to vars

## Accessing Fixture Info

Find out what other fixtures are requested:

In [17]:
import pytest
  
@pytest.fixture
def db(request):
    if 'transactional_db' in request.fixturenames:
        pytest.fail('Conflicting fixtures')
    return no_transactions_db()
  
@pytest.fixture
def transactional_db(request):
    if 'db' in request.fixturenames:
        pytest.fail('Conflicting fixtures')
    return transactional_db()


- Mutual exclusive fixture

## Plugins and Hooks

```
myproj/
+- myproj/
|  +- __init__.py
|  +- models.py
+- tests/
   +- contest.py
   +- test_models.py
```

### A few common hooks:

* `pytest_addoption(parser)`
* `pytest_ignore_collect(path, config)`
* `pytest_sessionstart(session)`
* `pytest_sessionfinish(session, exitstatus)`
* `pytest_assertrepr_compare(config, op, left, right)`

See hookspec for full list


- `conftest.py` is a plugin
- For advanced test suites you end up writing your own plugin.
- Source in hookspec.py or in docs
- Arguments are optional


## Using commandline options

New options an be accessed from fixtures and tests:

In [18]:
#conftest.py
import pytest
def pytest_addoption(parser):
    parser.addoption('--ci', action='store_true',
                     help='Indicate tests are run on CI server')
@pytest.fixture
def fix(request):
    ci = request.config.getoption('ci')

# Test module
def test_foo(pytestconfig):
    ci = pytestconfig.getoption('ci')



- You can add your own command line options


## skip or fail

Skipping not allowed on CI server:

In [19]:
import pytest

@pytest.fixture(scope='session')
def redis_client(request):
    servers = ['localhost', 'venera.clockhouse']
    for hostname in servers:
        try:
            return redis.StrictRedis(hostname)
        except redis.ConnectionError:
            continue
        else:
            if request.config.getoption('ci'):
                pytest.fail('No Redis server found')
            else:
                pytest.skip('No Redis server found')


- Also mention to pass server loc via cmd

## Questions?

Thanks for listening!